In [1]:
import numpy as np
import sys
import pathlib
from os.path import join
path_to_file = str(pathlib.Path().resolve())
dir_path = join(path_to_file)

sys.path.append(join(dir_path, "HelperFiles"))
from helper import *
from rankshap import *
from train_models import *
from load_data import *

from os.path import join
data_path = join(dir_path, "Experiments", "Data")

import warnings
warnings.filterwarnings('ignore')

/Users/jeremygoldwasser/opt/anaconda3/envs/shap/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load dataset and train model

In [2]:
dataset = "census"
# X_train, y_train, X_test, y_test, mapping_dict = make_data(data_path, dataset)
# save_data(data_path, dataset, X_train, y_train, X_test, y_test, mapping_dict)
X_train, y_train, X_test, y_test, mapping_dict = load_data(data_path, dataset)

# model = train_neural_net(X_train, y_train)
model = train_logreg(X_train, y_train)

print("Class imbalance: {}%".format(round(100*(max(np.mean(y_test), 1-np.mean(y_test))))))
Y_preds = (model(X_test) > 0.5).astype("int")
print("NN {}% accuracy".format(round(np.mean(Y_preds == y_test)*100)))



Class imbalance: 75%
NN 84% accuracy


## Preliminary run of Shapley sampling

Not infrequently, expected number of samples to be significant exceeds max number of perms; sampling it, though, it's significant. This suggests our distribution is too conservative. Maybe we can remove the factor of 2.

In [11]:
np.random.seed(1)
xloc = X_test[0:1]
shap_vals, diffs_all_feats = rankshap(model, X_train, xloc, K=2, alpha=0.2, mapping_dict=mapping_dict, abs=True)
print(get_ordering(shap_vals, abs=True))



[ 3  2  4  8  7  5  0  1  9 10 11  6]


# Simulate Experimental FWER

In [15]:
# xloc = X_test[7:8]
N_runs = 50
K = 2
shap_vals_all = []
top_K = []
while len(top_K) < N_runs:
    shap_vals, diffs_all_feats, converged = rankshap(model, X_train, xloc, K=3, alpha=0.2, max_n_perms=10000, mapping_dict=mapping_dict, abs=True)
    if converged:
        est_top_K = get_ordering(shap_vals, abs=True)[:K]
        top_K.append(est_top_K)
        shap_vals_all.append(shap_vals)
        if len(top_K) % 5 == 0: print(calc_fwer(top_K))

Hit max # perms without converging. Returning `NA`.
Hit max # perms without converging. Returning `NA`.
Hit max # perms without converging. Returning `NA`.
Hit max # perms without converging. Returning `NA`.


KeyboardInterrupt: 

[]